# 02: Structured Outputs with Pydantic

**Duration:** 30 minutes

**What You'll Learn:**
- Why structured outputs matter
- How to use Pydantic models to define schemas
- Getting JSON instead of free text from LLMs
- Handling validation and retries

**Why This Matters:**
Free text responses are great for humans, but terrible for software. Structured outputs let you build reliable systems that process LLM responses programmatically.

---

## Step 1: Install Pydantic

Pydantic is Python's most popular data validation library. It lets you define data structures with types and validation rules.

In [12]:
!pip install httpx pydantic

## Step 2: Define Your First Pydantic Model

Instead of getting "RELEVANT" as text, let's get a structured classification result.

In [13]:
from pydantic import BaseModel, Field
from typing import List
from enum import Enum

class TenderCategory(str, Enum):
    """Possible tender categories"""
    CYBERSECURITY = "cybersecurity"
    AI = "ai"
    SOFTWARE = "software"
    OTHER = "other"

class TenderClassification(BaseModel):
    """Structured output for tender classification"""
    is_relevant: bool = Field(description="Is this tender relevant?")
    confidence: float = Field(description="Confidence score 0-1", ge=0, le=1)
    categories: List[TenderCategory] = Field(description="Detected categories")
    reasoning: str = Field(description="Why you made this decision")

# Test creating an instance
example = TenderClassification(
    is_relevant=True,
    confidence=0.95,
    categories=[TenderCategory.CYBERSECURITY, TenderCategory.AI],
    reasoning="This tender combines cybersecurity and AI which matches our expertise."
)

print("Example classification:")
print(example.model_dump_json(indent=2))

Example classification:
{
  "is_relevant": true,
  "confidence": 0.95,
  "categories": [
    "cybersecurity",
    "ai"
  ],
  "reasoning": "This tender combines cybersecurity and AI which matches our expertise."
}


## Step 3: Understanding JSON Schema

Pydantic models can export JSON schemas. We'll use this to tell the LLM what format we want.

In [14]:
import json

# Get the JSON schema from our model
schema = TenderClassification.model_json_schema()

print("JSON Schema for TenderClassification:")
print(json.dumps(schema, indent=2))

# This tells us:
# - What fields are required
# - What type each field should be
# - Validation constraints (like confidence between 0 and 1)

JSON Schema for TenderClassification:
{
  "$defs": {
    "TenderCategory": {
      "description": "Possible tender categories",
      "enum": [
        "cybersecurity",
        "ai",
        "software",
        "other"
      ],
      "title": "TenderCategory",
      "type": "string"
    }
  },
  "description": "Structured output for tender classification",
  "properties": {
    "is_relevant": {
      "description": "Is this tender relevant?",
      "title": "Is Relevant",
      "type": "boolean"
    },
    "confidence": {
      "description": "Confidence score 0-1",
      "maximum": 1,
      "minimum": 0,
      "title": "Confidence",
      "type": "number"
    },
    "categories": {
      "description": "Detected categories",
      "items": {
        "$ref": "#/$defs/TenderCategory"
      },
      "title": "Categories",
      "type": "array"
    },
    "reasoning": {
      "description": "Why you made this decision",
      "title": "Reasoning",
      "type": "string"
    }
  },
  "requ

## Step 4: Building the Structured Prompt

To get structured output, we need to tell the LLM:
1. What format we want (JSON)
2. What the schema looks like
3. That it should ONLY return valid JSON

In [15]:
def build_structured_prompt(user_prompt: str, model_class: type[BaseModel]) -> str:
    """Add schema instructions to a prompt"""
    
    # Get an example of the output format
    schema = model_class.model_json_schema()
    
    # Build complete prompt with instructions
    full_prompt = f"""{user_prompt}

CRITICAL: Respond with ONLY valid JSON matching this schema:
{json.dumps(schema, indent=2)}

Do not include any markdown formatting, code blocks, or explanatory text.
Return ONLY the raw JSON object.
"""
    
    return full_prompt

# Test it
test_prompt = "Is this tender about AI?"
structured = build_structured_prompt(test_prompt, TenderClassification)
print("Structured prompt:")
print(structured)

Structured prompt:
Is this tender about AI?

CRITICAL: Respond with ONLY valid JSON matching this schema:
{
  "$defs": {
    "TenderCategory": {
      "description": "Possible tender categories",
      "enum": [
        "cybersecurity",
        "ai",
        "software",
        "other"
      ],
      "title": "TenderCategory",
      "type": "string"
    }
  },
  "description": "Structured output for tender classification",
  "properties": {
    "is_relevant": {
      "description": "Is this tender relevant?",
      "title": "Is Relevant",
      "type": "boolean"
    },
    "confidence": {
      "description": "Confidence score 0-1",
      "maximum": 1,
      "minimum": 0,
      "title": "Confidence",
      "type": "number"
    },
    "categories": {
      "description": "Detected categories",
      "items": {
        "$ref": "#/$defs/TenderCategory"
      },
      "title": "Categories",
      "type": "array"
    },
    "reasoning": {
      "description": "Why you made this decision",
 

## Step 5: Making the API Call

Now let's call the LLM and parse the response into our Pydantic model.

In [16]:
import httpx
import json
from typing import TypeVar, Type

T = TypeVar('T', bound=BaseModel)

BASE_URL = "http://localhost:1234/v1"
MODEL = "local-model"

async def call_llm_structured(
    prompt: str, 
    response_model: Type[T],
    temperature: float = 0.1
) -> T:
    """Call LLM and return structured output"""
    
    # Build structured prompt
    full_prompt = build_structured_prompt(prompt, response_model)
    
    # Make API call
    async with httpx.AsyncClient(timeout=60.0) as client:
        response = await client.post(
            f"{BASE_URL}/chat/completions",
            json={
                "model": MODEL,
                "messages": [
                    {"role": "system", "content": "You are a precise data analyst. Always return valid JSON."},
                    {"role": "user", "content": full_prompt}
                ],
                "temperature": temperature,
            },
        )
        
        result = response.json()
        content = result["choices"][0]["message"]["content"]
        
        # Clean the response (remove markdown, extra text, etc.)
        content = content.strip()
        if content.startswith("```json"):
            content = content[7:]
        if content.startswith("```"):
            content = content[3:]
        if content.endswith("```"):
            content = content[:-3]
        content = content.strip()
        
        # Parse JSON and validate with Pydantic
        data = json.loads(content)
        return response_model.model_validate(data)

print("✓ Structured LLM function ready!")

✓ Structured LLM function ready!


## Step 6: Test with Real Data

Let's classify some real tenders using structured outputs.

In [17]:
# Example 1: Clearly relevant tender
print("Example 1: AI Cybersecurity Tender")
print("=" * 60)

prompt1 = """
Analyze this tender:

TITLE: AI-Powered Network Intrusion Detection System
DESCRIPTION: Government agency seeks vendor to develop and deploy machine learning-based 
cybersecurity solution for real-time threat detection across enterprise network. 
Must include automated incident response and integration with existing SIEM.

Determine if this is relevant for a tech company specializing in AI and cybersecurity.
"""

result1 = await call_llm_structured(prompt1, TenderClassification)
print(f"Relevant: {result1.is_relevant}")
print(f"Confidence: {result1.confidence}")
print(f"Categories: {[c.value for c in result1.categories]}")
print(f"Reasoning: {result1.reasoning}")
print()

Example 1: AI Cybersecurity Tender
Relevant: True
Confidence: 0.95
Categories: ['cybersecurity', 'ai']
Reasoning: The tender requests a machine learning-based cybersecurity solution for real-time threat detection and automated incident response, directly aligning with the company's expertise in AI and cybersecurity.



In [18]:
# Example 2: Clearly NOT relevant
print("Example 2: Office Furniture")
print("=" * 60)

prompt2 = """
Analyze this tender:

TITLE: Office Furniture Supply Contract
DESCRIPTION: Supply 500 ergonomic chairs, standing desks, and filing cabinets 
for new government office building. Delivery required within 3 months.

Determine if this is relevant for a tech company specializing in AI and cybersecurity.
"""

result2 = await call_llm_structured(prompt2, TenderClassification)
print(f"Relevant: {result2.is_relevant}")
print(f"Confidence: {result2.confidence}")
print(f"Categories: {[c.value for c in result2.categories]}")
print(f"Reasoning: {result2.reasoning}")
print()

Example 2: Office Furniture
Relevant: False
Confidence: 0.95
Categories: ['other']
Reasoning: The tender requests ergonomic chairs, standing desks, and filing cabinets for a government office building. This procurement is purely for physical office furniture and does not involve any technology, software, AI solutions, or cybersecurity services. Therefore it is not relevant to a tech company specializing in AI and cybersecurity.



In [19]:
# Example 3: Edge case - Hardware with some software
print("Example 3: Hardware with Software Component")
print("=" * 60)

prompt3 = """
Analyze this tender:

TITLE: Firewall Hardware Procurement
DESCRIPTION: Purchase 50 enterprise firewall appliances. Vendor must provide 
installation and basic configuration. Devices come with built-in software.

Determine if this is relevant for a tech company specializing in AI and cybersecurity.
"""

result3 = await call_llm_structured(prompt3, TenderClassification)
print(f"Relevant: {result3.is_relevant}")
print(f"Confidence: {result3.confidence}")
print(f"Categories: {[c.value for c in result3.categories]}")
print(f"Reasoning: {result3.reasoning}")

Example 3: Hardware with Software Component
Relevant: True
Confidence: 0.95
Categories: ['cybersecurity']
Reasoning: The tender seeks enterprise firewall appliances with installation and configuration, directly pertaining to cybersecurity infrastructure. An AI-focused company that also specializes in cybersecurity would find this procurement relevant as it aligns with their core domain of securing networks and could integrate AI-driven threat detection or management into the firewall solutions.


## Step 7: Handling Validation Errors

Sometimes LLMs return invalid data. Pydantic catches these errors!

In [20]:
from pydantic import ValidationError

# Try to create invalid data
print("Testing validation:")
print("=" * 60)

try:
    # This should fail - confidence > 1.0
    bad_data = TenderClassification(
        is_relevant=True,
        confidence=1.5,  # Invalid! Must be 0-1
        categories=[TenderCategory.AI],
        reasoning="Test"
    )
except ValidationError as e:
    print("✓ Caught validation error:")
    print(e)

print("\n")

try:
    # This should fail - wrong enum value
    bad_data2 = TenderClassification(
        is_relevant=True,
        confidence=0.8,
        categories=["blockchain"],  # Invalid category!
        reasoning="Test"
    )
except ValidationError as e:
    print("✓ Caught enum error:")
    print(e)

Testing validation:
✓ Caught validation error:
1 validation error for TenderClassification
confidence
  Input should be less than or equal to 1 [type=less_than_equal, input_value=1.5, input_type=float]
    For further information visit https://errors.pydantic.dev/2.12/v/less_than_equal


✓ Caught enum error:
1 validation error for TenderClassification
categories.0
  Input should be 'cybersecurity', 'ai', 'software' or 'other' [type=enum, input_value='blockchain', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/enum


## Step 8: Adding Retry Logic

LLMs are unreliable. Sometimes they return malformed JSON. Let's add retries.

In [21]:
import asyncio

async def call_llm_structured_with_retry(
    prompt: str,
    response_model: Type[T],
    max_retries: int = 3,
    temperature: float = 0.1
) -> T:
    """Call LLM with retry logic for robustness"""
    
    for attempt in range(max_retries):
        try:
            result = await call_llm_structured(prompt, response_model, temperature)
            if attempt > 0:
                print(f"  ✓ Success on attempt {attempt + 1}")
            return result
            
        except (json.JSONDecodeError, ValidationError, KeyError) as e:
            print(f"  Attempt {attempt + 1}/{max_retries} failed: {str(e)[:100]}")
            
            if attempt == max_retries - 1:
                raise Exception(f"Failed after {max_retries} attempts: {e}")
            
            # Wait before retry
            await asyncio.sleep(1)

print("✓ Retry logic ready!")

✓ Retry logic ready!


## Step 9: Comparing Text vs Structured Outputs

Let's see the difference side by side.

In [22]:
async def call_llm_text(prompt: str) -> str:
    """Simple text response (from notebook 01)"""
    async with httpx.AsyncClient(timeout=60.0) as client:
        response = await client.post(
            f"{BASE_URL}/chat/completions",
            json={
                "model": MODEL,
                "messages": [{"role": "user", "content": prompt}],
                "temperature": 0.1,
            },
        )
        result = response.json()
        return result["choices"][0]["message"]["content"]

test_prompt = """
Is this tender relevant for a tech company?

TITLE: Custom Software Development for Tax Portal
DESCRIPTION: Build web application for tax filing with secure authentication.

Answer with yes/no and explain briefly.
"""

print("TEXT OUTPUT:")
print("=" * 60)
text_result = await call_llm_text(test_prompt)
print(text_result)
print(f"\nType: {type(text_result)}")
print("Hard to parse programmatically\n\n")

print("STRUCTURED OUTPUT:")
print("=" * 60)
structured_result = await call_llm_structured_with_retry(test_prompt, TenderClassification)
print(structured_result.model_dump_json(indent=2))
print(f"\nType: {type(structured_result)}")
print(f"Easy to use: structured_result.is_relevant = {structured_result.is_relevant}")
print(f"Type-safe: structured_result.confidence = {structured_result.confidence}")

TEXT OUTPUT:
**Yes**

The tender is specifically for developing a custom web application (a tax portal) with secure authentication, which falls squarely within the core services of a tech company that builds software solutions. It involves web development, security implementation, and integration with tax‑related data—areas that a tech firm would typically handle.

Type: <class 'str'>
Hard to parse programmatically


STRUCTURED OUTPUT:
{
  "is_relevant": true,
  "confidence": 0.95,
  "categories": [
    "software"
  ],
  "reasoning": "The tender requests the development of a web application for tax filing, which directly involves custom software creation and aligns with the services offered by a tech company."
}

Type: <class '__main__.TenderClassification'>
Easy to use: structured_result.is_relevant = True
Type-safe: structured_result.confidence = 0.95


## 🎉 Congratulations!

You now understand structured outputs!

## Key Takeaways

1. **Pydantic models define schemas** - Clear contract for LLM outputs
2. **JSON schemas guide LLMs** - Tell them exactly what format you need
3. **Validation catches errors** - Pydantic ensures data quality
4. **Retries handle failures** - LLMs are unreliable, plan for it
5. **Structured > Text** - Much easier to build software with

## Pattern You Learned

```python
# 1. Define schema
class MyOutput(BaseModel):
    field1: str
    field2: float

# 2. Add schema to prompt
prompt = f"{user_question}\n\nReturn JSON matching: {schema}"

# 3. Parse and validate
result = MyOutput.model_validate(json.loads(llm_response))

# 4. Use type-safe data
print(result.field1)  # IDE knows this is a string!
```

## Next Steps

Now that you can get structured outputs, let's build our first agent: the **Filter Agent** that classifies tenders!

➡️ Continue to `03_filter_agent.ipynb`